In [12]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time

dataset = pd.DataFrame({
    'City': ['Berlin', 'Dakar', 'Kananga'],
    'State': ['Berlin', 'Dakar', 'Kasai-Occidental'],
    'Country': ['Germany', 'Senegal', 'Democratic Republic of the Congo'],
    'Lon': [np.nan, np.nan, np.nan],
    'Lat': [np.nan, np.nan, np.nan]
})


def get_info(x):
    time.sleep(1.2)
    if pd.notnull(x['Lat']) and pd.notnull(x['Lon']):
        return x['Lat'], x['Lon']
    address = x['City'] + ', ' + x['State'] + ', ' + x['Country']
    geolocator = geolocator = Nominatim(user_agent='PowerBIGeo')
    location = geolocator.geocode(query=address)
    if location:
        return location.latitude, location.longitude
    else:
        address = x['City'] + ', ' + x['Country']
        location = geolocator.geocode(query=address)
        if location:
            return location.latitude, location.longitude
        return pd.NA, pd.NA
    
    
dataset[['Lat', 'Lon']] = dataset.apply(get_info, axis=1, result_type='expand')

In [9]:
dataset

,City,State,Country,Lon,Lat
0,Berlin,Berlin,Germany,13.338652,52.503379
1,Dakar,Dakar,Senegal,-17.447938,14.693425
2,Kananga,Kasai-Occidental,Democratic Republic of the Congo,22.408589,-5.895173
